# Convergence 2D elasticity to beam
We investigate numerically how good the Timoshenko beam and Bernoulli beam model approximate the full 2D elasticity problem for thickness $t\to0$.

In [ ]:
from ngsolve import *
from ngsolve.meshes import Make1DMesh, MakeStructured2DMesh
from ngsolve.webgui import Draw

We choose material parameters such that the elasticity tensor $\mathbb{C}$ becomes the identity. We consider a beam of length $1$ and thickness $t$, which is fixed on the left and apply a shear force at the right boundary.

In [ ]:
# use material parameters such that equations simplify
mu, lam = 0.5, 0  # Lame parameter
nu = lam/(2*(lam+mu)) # Possion ratio (=0)
E = mu*(3*lam+2*mu)/(lam+mu) # Young's modulus (=1)
k = 5/6           # shear correction factor
G = E/(2*(1+nu))  # shearing modulus
force = CF( (0,-0.1) )

mesh = MakeStructured2DMesh(nx=10, ny=1, mapping=lambda x,y : (x,0.1*y))
Draw(mesh);

Solve the 2D elasticity problem with the TDNNS method, which does not suffer from shear locking.

In [ ]:
def SolveTDNNS(order, mesh, t, draw=False):
    fesU = HCurl(mesh, order=order, dirichlet="left")
    fesS = HDivDiv(mesh, order=order,dirichlet="top|bottom|right")
    X = fesS*fesU
    (sigma,u),(tau,v) = X.TnT()
            
    n = specialcf.normal(2)  
    def tang(u): return u-(u*n)*n
            
    a = BilinearForm(X, symmetric=True, symmetric_storage=True)
    a += (InnerProduct(sigma,tau) + div(sigma)*v + div(tau)*u -1e-10*u*v)*dx
    a += (-(sigma*n)*tang(v) - (tau*n)*tang(u) )*dx(element_boundary=True)
    a.Assemble()
            
    f = LinearForm(X)
    f += -t**2*force*v.Trace()*ds("right")
    f.Assemble()
            
    gfsol = GridFunction(X)
    _, gfu = gfsol.components
    
    gfsol.vec.data = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")*f.vec
    
    if draw:
        Draw(gfu, mesh, "u", deformation=True)
    
    return gfu
t=1e-2
mesh2D = MakeStructured2DMesh(nx=10,ny=1, mapping=lambda x,y : (x,t*(y-0.5)))
gfuTDNNS = SolveTDNNS(4, t=t, mesh=mesh2D, draw=True)

Solve the Timoshenko beam and Euler-Bernoulli beam problem.

In [ ]:
def SolveTB(order, mesh, t, reduced_integration=False, draw=False):
    V = H1(mesh, order=order, dirichlet="left")
    fes = V*V    
    (u,beta),(du,dbeta) = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True)
    a += 1/12*grad(beta)*grad(dbeta)*dx + k*G/t**2*(grad(u)-beta)*(grad(du)-dbeta)*dx(bonus_intorder=-reduced_integration)
    
    f = LinearForm(fes)
    f += force[1]*du*ds("right")
    
    gfsol = GridFunction(fes)
    
    a.Assemble()
    f.Assemble()
    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec
    gfu, gfbeta = gfsol.components
    
    if draw:
        Draw(gfu, mesh, deformation=CF((0, gfu, 0)))
    
    return gfu, gfbeta

mesh1D = Make1DMesh(20)

gfuTB, gfbetaTB = SolveTB(order=3, mesh=mesh1D, t=t, draw=True)

def SolveBB(order, mesh, draw=False):
    fes = H1(mesh, order=order, dirichlet="left")*H1(mesh, order=order, dirichlet="right")
    (u,sigma),(du,dsigma) = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True)
    a += (12*sigma*dsigma + grad(u)*grad(dsigma) + grad(du)*grad(sigma) -1e-10*u*du)*dx
    
    f = LinearForm(fes)
    f += -force[1]*du*ds("right")
    
    gfsol = GridFunction(fes)
    
    a.Assemble()
    f.Assemble()
    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec

    gfw, _ = gfsol.components
    
    if draw:
        Draw(gfw, mesh, deformation=CF((0, gfw, 0)))
    return gfw

gfuBB = SolveBB(order=3, mesh=Make1DMesh(20), draw=True)

Now we solve the 2D elasticity and Timoschenko beam problem for different thicknesses $t$ (the Bernoulli beam is independent of $t$ and gets thus only solved once) and compute the relative error of the beam models with respect to the 2D elasticity solution by comparing the vertical deflection at the midsurface at the right boundary.

In [ ]:
results = []
gfwBB = SolveBB(order=1, mesh=Make1DMesh(200))
gfuBB = CF( (-y*Grad(gfwBB), gfwBB) )

with TaskManager():
    for t in [10**(-i) for i in range(6)]:
        mesh2D = MakeStructured2DMesh(nx=int(50),ny=1, mapping=lambda x,y : (x,t*(y-0.5)))
        gfuTDNNS = SolveTDNNS(1,t=t,mesh=mesh2D)
        l2TDNNS = sqrt(Integrate(gfuTDNNS*gfuTDNNS,mesh2D))
        gfwTB, gfbetaTB = SolveTB(order=1, mesh=Make1DMesh(200), reduced_integration=True, t=t)
        gfuTB = CF( (-y*gfbetaTB,gfwTB) )
        results.append((t, sqrt(Integrate((gfuTDNNS-gfuTB)**2,mesh2D))/l2TDNNS, \
                  sqrt(Integrate((gfuTDNNS-gfuBB)**2,mesh2D))/l2TDNNS))

In [ ]:
import matplotlib.pyplot as plt
plt.yscale('log')
plt.xscale('log')
plt.xlabel("t")
plt.ylabel("relative err")
ts, errtb, errbb = zip(*results)
plt.plot(ts, errtb, '*-', label="$|u_{\mathrm{TB}}-u_{\mathrm{2D}}|$")
plt.plot(ts, errbb, 'x-', label="$|u_{\mathrm{BB}}-u_{\mathrm{2D}}|$")
plt.plot(ts, [th for th in ts], '-' , color="k", label="$O(t)$")
plt.plot(ts, [th**2 for th in ts], '-.' , color="k", label="$O(t^2)$")
plt.legend()
plt.show()

We observe linear and quadratic convergence for the Euler-Bernoulli and Timoshenko beam, respectively. However, the 2D reference solution becomes unstable for extremely small thicknesses. Therefore, let's investigate the convergence of the Euler-Bernoulli beam to the Timoshenko beam instead.

In [ ]:
results2 = []
mesh1d = Make1DMesh(100)
gfwBB = SolveBB(order=2, mesh=mesh1d)
l2bb = sqrt(Integrate(gfwBB**2,mesh1d))

with TaskManager():
    for t in [10**(-i) for i in range(6)]:
        gfwTB, _ = SolveTB(order=2, mesh=mesh1d, t=t)
        results2.append((t, sqrt(Integrate((gfwBB-gfwTB)**2,mesh1d))/l2bb))

Again, one of the solutions becomes unstable (the Timoshenko beam because of the $1/t^2$ term). Thus, replace the numerical solution of the Timoshenko beam with its exact one.

In [ ]:
plt.yscale('log')
plt.xscale('log')
plt.xlabel("t")
plt.ylabel("relative err")
ts, errtb = zip(*results2)
plt.plot(ts, errtb, '*-', label="$|u_{\mathrm{TB}}-u_{\mathrm{BB}}|$")
plt.plot(ts, [th**2 for th in ts], '-.' , color="k", label="$O(t^2)$")
plt.legend()
plt.show()

In [ ]:
from sympy import symbols, solve, lambdify
def ComputeExactSolutionTB(t=0.01, q=-0.1, shear_force=True):
    alpha = 1/(12*k*G)
    X, C1, C2, C3, C4 = symbols("x C1 C2 C3 C4")
    if shear_force:
        w_ex = -C1/6*X**3-C2/2*X**2 + C3*X+C4
        w_diff_ex = -C1/2*X**2-C2*X + C3
        beta_ex = -C1/2*X**2-C2*X-alpha*t**2*C1+C3
        beta_diff_ex = -C1*X-C2

        solve_bc = solve((w_ex.subs(X,0), (k*G/t**2*(w_diff_ex-beta_ex)-q).subs(X,1), beta_ex.subs(X,0), beta_diff_ex.subs(X,1)), C1, C2, C3, C4)
    else: # uniform volume force
        w_ex = q/2*X**4-C1/6*X**3-0.5*(C2+alpha*t**2*q*12)*X**2+C3*X+C4
        w_diff_ex = 2*q*X**3-C1/2*X**2-(C2+alpha*t**2*q*12)*X+C3
        beta_ex = 2*q*X**3-C1/2*X**2-C2*X-alpha*t**2*C1+C3
        beta_diff_ex = q*12/2*X**2-C1*X-C2
        solve_bc = solve((w_ex.subs(X,0), (w_diff_ex-beta_ex).subs(X,1), beta_ex.subs(X,0), beta_diff_ex.subs(X,1)), C1, C2, C3, C4)
    c1,c2,c3,c4 = solve_bc[C1], solve_bc[C2], solve_bc[C3], solve_bc[C4]

    # generate CoefficientFunctions
    w_ex_func = lambdify((X), w_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
    cf_w_ex = w_ex_func(x)
    beta_ex_func = lambdify((X), beta_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
    cf_beta_ex = beta_ex_func(x)
    
    return (cf_w_ex, cf_beta_ex)

In [ ]:
results3 = []
mesh1d = Make1DMesh(100)
gfwBB = SolveBB(order=2, mesh=mesh1d)
l2bb = sqrt(Integrate(gfwBB**2,mesh1d))

with TaskManager():
    for t in [10**(-i) for i in range(6)]:
        exTB, _ = ComputeExactSolutionTB(t=t, q=-0.1, shear_force=True)
        results3.append((t, sqrt(Integrate((gfwBB-exTB)**2,mesh1d))/l2bb))

Now we obtain a nice quadratic convergence up to the discretization error of the Euler-Bernoulli beam (use finer meshes or higher polynomial order to increase the accuracy).

In [ ]:
plt.yscale('log')
plt.xscale('log')
plt.xlabel("t")
plt.ylabel("relative err")
ts, errtb = zip(*results3)
plt.plot(ts, errtb, '*-', label="$|u_{\mathrm{TB}}-u_{\mathrm{BB}}|$")
plt.plot(ts, [th**2 for th in ts], '-.' , color="k", label="$O(t^2)$")
plt.legend()
plt.show()